## Extract FL Data

In [7]:
import numpy as np
import pandas as pd

In [8]:
train_df = pd.read_csv('../../data_analysis/fd001/fd001-scaled_train.csv', sep=' ')
test_df = pd.read_csv('../../data_analysis/fd001/fd001-scaled_test.csv', sep=' ')
train_labels_df = pd.read_csv('../../data_analysis/fd001/fd001-training_labels.csv', sep=' ')
test_labels_df = pd.read_csv('../../data_analysis/fd001/fd001-testing_labels.csv', sep=' ')

In [9]:
columns = train_df.columns
ms_used = ['RUL']
for i in range(1, 22):
    if i not in [1, 5, 6, 9, 10, 14, 16, 18, 19]:
        ms_used.append('SensorMeasure' + str(i))

In [10]:
train_df['RUL'] = train_labels_df['RUL']
test_df['RUL'] = test_labels_df['RUL']

In [11]:
train_gb = train_df.groupby(['ID'], sort = False)
test_gb = test_df.groupby(['ID'], sort = False)

In [12]:
# Args: List of pcts -> ex: [0.75, 0.2, 0.05]
def split_data(pcts, mode = 'train'):

    gb = train_gb if mode == 'train' else test_gb

    idx_remaining = [i for i in range(1, len(gb) + 1)]
    nums = []
    indices_concat = []
    for pct in pcts:
        nums.append(round(pct * len(gb)))
    n_workers = len(pcts)

    for i in range(n_workers):
        idx_worker = np.sort(np.random.choice(idx_remaining, nums[i], replace = False))
        indices_concat.append(idx_worker) 
        idx_remaining = np.setdiff1d(idx_remaining, idx_worker)

    for i in range(len(indices_concat)):
        id_list = indices_concat[i]
        df = gb.get_group(id_list[0])
        for j in range(1, len(id_list)):
            df = pd.concat([df, gb.get_group(id_list[j])])
        df.to_csv('./data/fd001/2 workers/' + mode + '_partition_' + str(i) + '.csv', sep=',', index = False)

pct_list = [0.5, 0.5]

split_data(pct_list, mode = 'train')
split_data(pct_list, mode = 'test')

### Test data - convert format

In [13]:
train_df

,ID,Cycle,SensorMeasure2,SensorMeasure3,SensorMeasure4,SensorMeasure7,SensorMeasure8,SensorMeasure11,SensorMeasure12,SensorMeasure13,SensorMeasure15,SensorMeasure17,SensorMeasure20,SensorMeasure21,RUL
0,1,1,-1.721725,-0.134255,-0.925936,1.121141,-0.516338,-0.266467,0.334262,-1.058890,-0.603816,-0.781710,1.348493,1.194427,191
1,1,2,-1.061780,0.211528,-0.643726,0.431930,-0.798093,-0.191583,1.174899,-0.363646,-0.275852,-0.781710,1.016528,1.236922,190
2,1,3,-0.661813,-0.413166,-0.525953,1.008155,-0.234584,-1.015303,1.364721,-0.919841,-0.649144,-2.073094,0.739891,0.503423,189
3,1,4,-0.661813,-1.261314,-0.784831,1.222827,0.188048,-1.539489,1.961302,-0.224597,-1.971665,-0.781710,0.352598,0.777792,188
4,1,5,-0.621816,-1.251528,-0.301518,0.714393,-0.516338,-0.977861,1.052871,-0.780793,-0.339845,-0.136018,0.463253,1.059552,187
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,1.618000,1.216258,2.188375,-2.189329,1.315066,1.980044,-2.607969,2.278282,1.425294,2.446751,-1.805173,-2.921113,4
20627,100,197,1.717992,2.279706,2.738351,-2.833345,1.878576,1.867718,-2.350355,1.722087,1.913240,1.155367,-2.856395,-1.203764,3
20628,100,198,1.478011,1.946971,2.138377,-2.742957,2.019453,2.054927,-1.902919,2.000184,3.265092,3.092444,-2.081810,-3.292481,2
20629,100,199,1.098043,2.403666,1.955051,-3.036719,2.160330,3.178182,-2.363913,1.861136,2.579834,1.155367,-2.911722,-2.085072,1
